## Homework 2 final version


In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_data(token):
    
    if token=="FLOW":
        platform="kraken"
    else:
        platform="coinbase-pro"
        
    res = requests.get(
    f'https://api.cryptowat.ch/markets/{platform}/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [3]:
tokens = ['ETH', 'SOL', 'AVAX', 'USDT','FLOW']

In [4]:
dfs = [
    (lambda x: x.assign(chain=np.where(x.token.isin(['ETH', 'AAVE', 'COMP']), np.full(x.shape[0], 'ETH'), x.token)))(get_data(token)) 
    for token in tokens
]

In [5]:
dfs[0]

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH,ETH
1,2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH,ETH
2,2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH,ETH
3,2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH,ETH
4,2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH,ETH
...,...,...,...,...,...,...,...,...,...
569,2021-12-15 17:00:00,3659.57,3724.17,3643.43,3715.95,26080.393747,9.600520e+07,ETH,ETH
570,2021-12-15 18:00:00,3716.82,3741.54,3693.38,3728.75,13429.938131,4.995921e+07,ETH,ETH
571,2021-12-15 19:00:00,3728.80,3843.25,3720.82,3810.54,19300.639196,7.292436e+07,ETH,ETH
572,2021-12-15 20:00:00,3811.38,3989.00,3760.98,3972.00,62360.886127,2.444909e+08,ETH,ETH


In [6]:
df_base = pd.concat(get_data(token) for token in tokens)

In [7]:
df_base.head()

,ts,open,high,low,close,volume,volumeUSD,token
0,2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH
1,2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH
2,2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH
3,2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH
4,2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH


In [8]:
new_df_volume = df_base[["volumeUSD","token"]]
new_df_volume

,volumeUSD,token
0,3.123077e+07,ETH
1,3.574586e+07,ETH
2,3.895639e+07,ETH
3,3.882032e+07,ETH
4,3.315476e+07,ETH
...,...,...
569,1.223179e+05,FLOW
570,6.152375e+03,FLOW
571,4.400994e+04,FLOW
572,1.938078e+05,FLOW


In [9]:
new_df_volume.set_index("token", inplace=True)

In [10]:
new_df_volume.sort_values('volumeUSD', ascending=False)

,volumeUSD
token,
ETH,3.702769e+08
ETH,2.444909e+08
ETH,2.147795e+08
ETH,1.973315e+08
ETH,1.846590e+08
...,...
FLOW,2.028609e+03
FLOW,1.741454e+03
FLOW,1.262889e+03


In [11]:
ETHdata=get_data("ETH")
SOLdata=get_data("SOL")

In [12]:
ratios=(ETHdata["close"]/SOLdata["close"])

In [13]:
ratios=ratios.to_frame()

In [14]:
ratios

,close
0,18.471694
1,18.736150
2,18.577985
3,18.794937
4,18.891059
...,...
569,22.997586
570,22.397585
571,22.542239
572,22.644091


In [15]:
dfs[("token"=="ETH") or ("token"=="SOL")]["ratios"]=ratios

In [16]:
dfs

[                     ts     open     high      low    close        volume  \
 0   2021-11-22 00:00:00  4317.98  4342.24  4246.07  4262.99   7262.562789   
 1   2021-11-22 01:00:00  4263.04  4270.34  4212.45  4234.37   8437.946084   
 2   2021-11-22 02:00:00  4234.87  4246.72  4171.17  4217.89   9259.725370   
 3   2021-11-22 03:00:00  4217.88  4223.48  4163.58  4193.47   9259.899519   
 4   2021-11-22 04:00:00  4192.95  4213.59  4147.00  4168.35   7934.546906   
 ..                  ...      ...      ...      ...      ...           ...   
 569 2021-12-15 17:00:00  3659.57  3724.17  3643.43  3715.95  26080.393747   
 570 2021-12-15 18:00:00  3716.82  3741.54  3693.38  3728.75  13429.938131   
 571 2021-12-15 19:00:00  3728.80  3843.25  3720.82  3810.54  19300.639196   
 572 2021-12-15 20:00:00  3811.38  3989.00  3760.98  3972.00  62360.886127   
 573 2021-12-15 21:00:00  3972.22  3999.00  3967.14  3995.06   2381.154397   
 
         volumeUSD token chain     ratios  
 0    3.123077e+07

In [17]:
df_base = pd.concat(dfs[i] for i in range(len(tokens)))

In [18]:
df_base.head()

,ts,open,high,low,close,volume,volumeUSD,token,chain,ratios
0,2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH,ETH,18.471694
1,2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH,ETH,18.736150
2,2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH,ETH,18.577985
3,2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH,ETH,18.794937
4,2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH,ETH,18.891059


In [19]:
df_base.tail()

,ts,open,high,low,close,volume,volumeUSD,token,chain,ratios
569,2021-12-15 17:00:00,8.243,8.270,8.231,8.246,14843.769471,122317.856879,FLOW,FLOW,NaN
570,2021-12-15 18:00:00,8.271,8.362,8.271,8.361,738.473554,6152.374585,FLOW,FLOW,NaN
571,2021-12-15 19:00:00,8.344,8.481,8.340,8.395,5228.013228,44009.944548,FLOW,FLOW,NaN
572,2021-12-15 20:00:00,8.386,8.670,8.346,8.665,22554.459438,193807.796422,FLOW,FLOW,NaN
573,2021-12-15 21:00:00,8.681,8.729,8.681,8.720,1297.939398,11305.500346,FLOW,FLOW,NaN


In [20]:
df_base.rename(
    columns={
        'volume':'volumeBase',
        'volumeUSD':'volumeTerm'
    }
)

,ts,open,high,low,close,volumeBase,volumeTerm,token,chain,ratios
0,2021-11-22 00:00:00,4317.980,4342.240,4246.070,4262.990,7262.562789,3.123077e+07,ETH,ETH,18.471694
1,2021-11-22 01:00:00,4263.040,4270.340,4212.450,4234.370,8437.946084,3.574586e+07,ETH,ETH,18.736150
2,2021-11-22 02:00:00,4234.870,4246.720,4171.170,4217.890,9259.725370,3.895639e+07,ETH,ETH,18.577985
3,2021-11-22 03:00:00,4217.880,4223.480,4163.580,4193.470,9259.899519,3.882032e+07,ETH,ETH,18.794937
4,2021-11-22 04:00:00,4192.950,4213.590,4147.000,4168.350,7934.546906,3.315476e+07,ETH,ETH,18.891059
...,...,...,...,...,...,...,...,...,...,...
569,2021-12-15 17:00:00,8.243,8.270,8.231,8.246,14843.769471,1.223179e+05,FLOW,FLOW,NaN
570,2021-12-15 18:00:00,8.271,8.362,8.271,8.361,738.473554,6.152375e+03,FLOW,FLOW,NaN
571,2021-12-15 19:00:00,8.344,8.481,8.340,8.395,5228.013228,4.400994e+04,FLOW,FLOW,NaN
572,2021-12-15 20:00:00,8.386,8.670,8.346,8.665,22554.459438,1.938078e+05,FLOW,FLOW,NaN


In [21]:
from functools import reduce

In [22]:
def df_merge(left, right):
    return pd.merge(left, right, on='ts', how='inner')

df_fat_table = reduce(
    df_merge, 
    [df_base[['ts', 'close']].rename(columns={'close': f'close_{token}'}) for df_base, token in zip(dfs, tokens)]
)

In [23]:
df_fat_table

,ts,close_ETH,close_SOL,close_AVAX,close_USDT,close_FLOW
0,2021-11-22 00:00:00,4262.99,230.785,128.67,1.0012,14.150
1,2021-11-22 01:00:00,4234.37,226.000,134.85,1.0013,13.621
2,2021-11-22 02:00:00,4217.89,227.037,132.54,1.0012,13.753
3,2021-11-22 03:00:00,4193.47,223.117,133.38,1.0011,13.652
4,2021-11-22 04:00:00,4168.35,220.652,136.29,1.0009,13.581
...,...,...,...,...,...,...
569,2021-12-15 17:00:00,3715.95,161.580,94.29,1.0005,8.246
570,2021-12-15 18:00:00,3728.75,166.480,95.61,1.0004,8.361
571,2021-12-15 19:00:00,3810.54,169.040,96.43,1.0004,8.395
572,2021-12-15 20:00:00,3972.00,175.410,99.28,1.0004,8.665


In [24]:
df_fat_table.set_index("ts",inplace=True)

In [25]:
df_fat_table

,close_ETH,close_SOL,close_AVAX,close_USDT,close_FLOW
ts,,,,,
2021-11-22 00:00:00,4262.99,230.785,128.67,1.0012,14.150
2021-11-22 01:00:00,4234.37,226.000,134.85,1.0013,13.621
2021-11-22 02:00:00,4217.89,227.037,132.54,1.0012,13.753
2021-11-22 03:00:00,4193.47,223.117,133.38,1.0011,13.652
2021-11-22 04:00:00,4168.35,220.652,136.29,1.0009,13.581
...,...,...,...,...,...
2021-12-15 17:00:00,3715.95,161.580,94.29,1.0005,8.246
2021-12-15 18:00:00,3728.75,166.480,95.61,1.0004,8.361
2021-12-15 19:00:00,3810.54,169.040,96.43,1.0004,8.395
